## 主题模型(semantic topic model)与聚类的区别

在数据分析中，经常会进行非监督学习的聚类算法，它可以对我们的特征数据进行非监督的聚类。而主题模型也是非监督的算法，目的是得到文本按照主题的概率分布。从这个方面来说，主题模型和普通的聚类算法有着相似性。

聚类算法根据样本特征的相似度将其聚类，比如，通过计算数据样本之间的欧式距离、曼哈顿距离以及余弦距离等聚类。

而主题模型是对文本中隐含主题的一种建模方法。比如从“双11”和“天猫”这两个词在对应的文本中有很大的主题相关度，但是如果使用词的特征进行聚类的话，很难找出找出两者的话题关联，因为聚类方法不能考虑到到隐含的主题。

那么如何找到隐含的主题呢？常用的方法一般都是基于统计学的生成方法。即假设以一定的概率选择了一个主题，然后以一定的概率选择当前主题的词。最后这些词组成了我们当前的文本。所有词的统计概率分布可以从语料库获得，具体如何以“一定的概率做出选择”，这就是各种具体的主题模型算法的任务了。

经典的话题模型有潜在语义索引(Latent Semantic Indexing)以及隐含狄利克雷分布(Latent Dirichlet Allocation，LDA)。前者计算效率较低，在大规模文本中计算主题耗费时间较大。目前主要使用LDA获取文本主题。

LDA主题模型，也称三层贝叶斯概率模型，包括词、主题及文档三层。利用文档中单词的共现关系将单词按主题聚类，得到“文档-主题”和“主题-单词”两个概率分布

使用LDA获取文档主题的流程

LDA的python实现：

https://github.com/blei-lab/onlineldavb/blob/master/onlineldavb.py



In [68]:
from gensim import corpora, models
import jieba.posseg as jp, jieba
# 文本集
texts = [
    '美国教练坦言，没输给中国女排，是输给了郎平'*2,
    '美国女排无缘四强，听听主教练的评价'*2,
    '中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术'*2,
    '为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了'*2,
    '跑了长途才知道，SUV和轿车之间的差距'*2,
    '家用的轿车买什么好，推荐一款可以跑长途的车型'*2,
    '7级大风,最低气温只有3度！'*2,
    '做好大风降温天气防范应对工作'*2,
    '北京迎来大风降温天气，发布寒潮蓝色预警'*2,
    '高盛料中国明年经济增长率将低于6%'*2,
    '香港明显陷入衰退，经济发展将停滞不前'*2,
    '全球经济并未步入衰退'*2
    ]
# 根据词性和停用词过滤条件
jieba.add_word('四强',1, 'n')
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
stopwords = ('没', '就', '知道', '是', '才', '听听', '坦言', '全面', '越来越', '评价', '放弃', '人','再次','什么','的','')  # 停词
# 分词
words_ls = []
for text in texts:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords]
    words_ls.append(words)
# 构造词典
dictionary = corpora.Dictionary(words_ls)

# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
corpus = [dictionary.doc2bow(words) for words in words_ls]

In [69]:
corpus

[[(0, 2), (1, 2), (2, 4), (3, 2)],
 [(1, 2), (4, 2), (5, 2), (6, 2)],
 [(0, 2), (3, 2), (4, 2), (7, 2), (8, 2), (9, 2), (10, 2)],
 [(11, 2), (12, 2), (13, 2), (14, 2), (15, 2)],
 [(12, 2), (14, 2), (15, 2), (16, 2), (17, 2)],
 [(13, 2), (14, 2), (15, 2), (17, 2), (18, 2), (19, 2), (20, 2)],
 [(21, 2), (22, 2)],
 [(21, 2), (23, 2), (24, 2), (25, 2), (26, 2), (27, 2)],
 [(21, 2), (24, 2), (27, 2), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2)],
 [(33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2)],
 [(36, 2), (38, 2), (40, 2), (41, 2), (42, 2)],
 [(38, 2), (40, 2), (43, 2), (44, 2), (45, 2)]]

In [70]:
# lda模型，num_topics设置主题的个数
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=4)
# 打印所有主题，每个主题显示5个词
for topic in lda.print_topics(num_words=5):
    print(topic)

(0, '0.096*"跑" + 0.095*"长途" + 0.090*"SUV" + 0.053*"轿车" + 0.053*"买"')
(1, '0.112*"经济" + 0.059*"将" + 0.059*"料" + 0.059*"衰退" + 0.059*"增长率"')
(2, '0.085*"大风" + 0.084*"天气" + 0.083*"降温" + 0.047*"经济" + 0.047*"衰退"')
(3, '0.091*"美国" + 0.089*"输给" + 0.050*"无缘" + 0.050*"女排" + 0.050*"主教练"')


In [71]:
# 主题推断
print(lda.inference(corpus))

(array([[ 0.25730935,  0.2508089 ,  0.25009677, 10.241785  ],
       [ 0.25407112,  0.25020924,  0.25011393,  8.245606  ],
       [14.242125  ,  0.25041103,  0.2501324 ,  0.25733072],
       [10.244555  ,  0.2501121 ,  0.2500877 ,  0.2552445 ],
       [10.244497  ,  0.25011238,  0.25008804,  0.2553021 ],
       [ 0.27499762,  0.25018725,  0.2501478 , 14.224667  ],
       [ 0.250148  ,  0.2506274 ,  0.27120617,  4.2280188 ],
       [ 0.2501009 ,  0.25039053, 12.248053  ,  0.25145543],
       [ 0.25011826,  0.25110638, 16.247677  ,  0.25109696],
       [ 0.25009716, 14.247124  ,  0.25267738,  0.25010166],
       [ 0.25013965,  0.27270824, 10.227007  ,  0.25014496],
       [ 0.25009632, 10.246008  ,  0.25379294,  0.25010276]],
      dtype=float32), None)


In [72]:
print(words_ls)

[['美国', '输给', '中国女排', '输给', '郎平', '美国', '输给', '中国女排', '输给', '郎平'], ['美国', '女排', '无缘', '主教练', '美国', '女排', '无缘', '主教练'], ['中国女排', '晋级', '世锦赛', '主教练', '郎平', '执教', '艺术', '中国女排', '晋级', '世锦赛', '主教练', '郎平', '执教', '艺术'], ['买', 'MPV', 'SUV', '跑', '长途', '买', 'MPV', 'SUV', '跑', '长途'], ['跑', '长途', 'SUV', '轿车', '差距', '跑', '长途', 'SUV', '轿车', '差距'], ['家用', '轿车', '买', '推荐', '跑', '长途', '车型', '家用', '轿车', '买', '推荐', '跑', '长途', '车型'], ['大风', '最低气温', '大风', '最低气温'], ['做好', '大风', '降温', '天气', '防范', '应对', '做好', '大风', '降温', '天气', '防范', '应对'], ['北京', '迎来', '大风', '降温', '天气', '发布', '寒潮', '蓝色', '北京', '迎来', '大风', '降温', '天气', '发布', '寒潮', '蓝色'], ['高盛', '料', '中国', '经济', '增长率', '将', '低于', '高盛', '料', '中国', '经济', '增长率', '将', '低于'], ['香港', '陷入', '衰退', '经济', '将', '香港', '陷入', '衰退', '经济', '将'], ['全球', '经济', '并未', '步入', '衰退', '全球', '经济', '并未', '步入', '衰退']]


In [73]:
print(dictionary.token2id)

{'中国女排': 0, '美国': 1, '输给': 2, '郎平': 3, '主教练': 4, '女排': 5, '无缘': 6, '世锦赛': 7, '执教': 8, '晋级': 9, '艺术': 10, 'MPV': 11, 'SUV': 12, '买': 13, '跑': 14, '长途': 15, '差距': 16, '轿车': 17, '家用': 18, '推荐': 19, '车型': 20, '大风': 21, '最低气温': 22, '做好': 23, '天气': 24, '应对': 25, '防范': 26, '降温': 27, '北京': 28, '发布': 29, '寒潮': 30, '蓝色': 31, '迎来': 32, '中国': 33, '低于': 34, '增长率': 35, '将': 36, '料': 37, '经济': 38, '高盛': 39, '衰退': 40, '陷入': 41, '香港': 42, '全球': 43, '并未': 44, '步入': 45}


In [74]:
print(lda)

LdaModel(num_terms=46, num_topics=4, decay=0.5, chunksize=2000)


In [75]:
for e, values in enumerate(lda.inference(corpus)[0]):
    print(texts[e])
    for ee, value in enumerate(values):
        print('\t主题%d推断值%.2f' % (ee, value))
    print()

美国教练坦言，没输给中国女排，是输给了郎平美国教练坦言，没输给中国女排，是输给了郎平
	主题0推断值0.26
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值10.24

美国女排无缘四强，听听主教练的评价美国女排无缘四强，听听主教练的评价
	主题0推断值0.25
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值8.25

中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术
	主题0推断值14.24
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值0.26

为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了
	主题0推断值10.24
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值0.26

跑了长途才知道，SUV和轿车之间的差距跑了长途才知道，SUV和轿车之间的差距
	主题0推断值10.24
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值0.26

家用的轿车买什么好，推荐一款可以跑长途的车型家用的轿车买什么好，推荐一款可以跑长途的车型
	主题0推断值0.28
	主题1推断值0.25
	主题2推断值0.25
	主题3推断值14.22

7级大风,最低气温只有3度！7级大风,最低气温只有3度！
	主题0推断值0.25
	主题1推断值0.25
	主题2推断值0.27
	主题3推断值4.23

做好大风降温天气防范应对工作做好大风降温天气防范应对工作
	主题0推断值0.25
	主题1推断值0.25
	主题2推断值12.25
	主题3推断值0.25

北京迎来大风降温天气，发布寒潮蓝色预警北京迎来大风降温天气，发布寒潮蓝色预警
	主题0推断值0.25
	主题1推断值0.25
	主题2推断值16.25
	主题3推断值0.25

高盛料中国明年经济增长率将低于6%高盛料中国明年经济增长率将低于6%
	主题0推断值0.25
	主题1推断值14.25
	主题2推断值0.25
	主题3推断值0.25

香港明显陷入衰退，经济发展将停滞不前香港明显陷入衰退，经济发展将停滞不前
	主题0推断值0.25
	主题1推断值0.27
	主题2推断值10.23
	主题3推断值0.25

全球经济并未步

In [76]:
test = '中国女排将在郎平的率领下向世界女排三大赛的三连冠发起冲击'
bow = dictionary.doc2bow([word.word for word in jp.cut(test) if word.flag in flags and word.word not in stopwords])
ndarray = lda.inference([bow])[0]
print(test)
for e, value in enumerate(ndarray[0]):
    print('\t主题%d推断值%.2f' % (e, value))

中国女排将在郎平的率领下向世界女排三大赛的三连冠发起冲击
	主题0推断值0.26
	主题1推断值1.25
	主题2推断值0.27
	主题3推断值3.22


In [79]:
word_id = dictionary.doc2idx(['长途'])[0]
print(lda.get_term_topics(word_id))
for i in lda.get_term_topics(word_id):
    print('【长途】与【主题%d】的关系值：%.2f%%' % (i[0], i[1]*100))

[(0, 0.085643046), (3, 0.034006134)]
【长途】与【主题0】的关系值：8.56%
【长途】与【主题3】的关系值：3.40%


In [82]:
#获取词与话题的关系
for word, word_id in dictionary.token2id.items():
    print(word, lda.get_term_topics(word_id, minimum_probability=1e-8))

中国女排 [(0, 0.037860468), (1, 0.0018822927), (2, 0.0003397225), (3, 0.036761053)]
美国 [(0, 0.0003680031), (1, 0.0018816735), (2, 0.000351564), (3, 0.081146866)]
输给 [(0, 0.00034486855), (1, 0.0038154526), (2, 0.00034440248), (3, 0.0786668)]
郎平 [(0, 0.037884258), (1, 0.0033130262), (2, 0.0003467184), (3, 0.034753174)]
主教练 [(0, 0.03786489), (1, 0.00044849122), (2, 0.00036694822), (3, 0.03952467)]
女排 [(0, 0.00033560183), (1, 0.00041576914), (2, 0.00033417836), (3, 0.039607354)]
无缘 [(0, 0.00033199386), (1, 0.00041097193), (2, 0.0003238418), (3, 0.03967897)]
世锦赛 [(0, 0.037766553), (1, 0.000404357), (2, 0.00032068702), (3, 0.00034236658)]
执教 [(0, 0.037705794), (1, 0.00041172944), (2, 0.00032634934), (3, 0.0003467933)]
晋级 [(0, 0.037724346), (1, 0.00041154417), (2, 0.0003237689), (3, 0.00034459634)]
艺术 [(0, 0.03770996), (1, 0.00041028037), (2, 0.00032848763), (3, 0.00034460388)]
MPV [(0, 0.037675746), (1, 0.00041735783), (2, 0.00032391655), (3, 0.00035297716)]
SUV [(0, 0.080007434), (1, 0.00043088

In [85]:
lda.show_topic(1, 50) #第一个参数代表主题编号，第二个代表topk的词

[('经济', 0.11169073),
 ('将', 0.059221223),
 ('料', 0.059138026),
 ('衰退', 0.059116445),
 ('增长率', 0.05909829),
 ('高盛', 0.059082128),
 ('低于', 0.059052587),
 ('中国', 0.059051078),
 ('全球', 0.059015628),
 ('并未', 0.05897034),
 ('步入', 0.058892712),
 ('蓝色', 0.013335192),
 ('输给', 0.013271864),
 ('降温', 0.013085573),
 ('发布', 0.012830067),
 ('郎平', 0.012544363),
 ('迎来', 0.011906071),
 ('天气', 0.011839613),
 ('北京', 0.011711137),
 ('大风', 0.01158334),
 ('寒潮', 0.0106949685),
 ('中国女排', 0.010220674),
 ('美国', 0.010219541),
 ('长途', 0.006898138),
 ('最低气温', 0.006886893),
 ('跑', 0.006811435),
 ('主教练', 0.0068107885),
 ('买', 0.006777153),
 ('轿车', 0.006752241),
 ('SUV', 0.0067460597),
 ('差距', 0.0067415377),
 ('MPV', 0.006695312),
 ('女排', 0.006689295),
 ('陷入', 0.006682838),
 ('香港', 0.0066785975),
 ('执教', 0.0066739316),
 ('防范', 0.006673251),
 ('晋级', 0.0066732247),
 ('无缘', 0.006671043),
 ('艺术', 0.0066684),
 ('做好', 0.0066630417),
 ('推荐', 0.0066554337),
 ('应对', 0.006650155),
 ('世锦赛', 0.0066456757),
 ('车型', 0.0066434667),


In [86]:
print('总概率:',round(sum(i[1] for i in lda.show_topic(0, 9999)),2))

总概率: 1.0


# 计算困惑度

In [92]:
import math
def perplexity(ldamodel, testset, dictionary, size_dictionary, num_topics):
    """calculate the perplexity of a lda-model"""
    print ('the info of this ldamodel: \n')
    print ('num of testset: %s; size_dictionary: %s; num of topics: %s'%(len(testset), size_dictionary, num_topics))
    prep = 0.0
    prob_doc_sum = 0.0
    topic_word_list = [] # store the probablity of topic-word:[(u'business', 0.010020942661849608),(u'family', 0.0088027946271537413)...]
    for topic_id in range(num_topics):
        topic_word = ldamodel.show_topic(topic_id, size_dictionary)
        dic = {}
        for word, probability in topic_word:
            dic[word] = probability
        topic_word_list.append(dic)
    doc_topics_list = [] #store the doc-topic tuples:[(0, 0.0006211180124223594),(1, 0.0006211180124223594),...]
    for doc in testset:
        doc_topics_list.append(ldamodel.get_document_topics(doc, minimum_probability=0))
    testset_word_num = 0
    for i in range(len(testset)):
        prob_doc = 0.0 # the probablity of the doc
        doc = testset[i]
        doc_word_num = 0 # the num of words in the doc
        for word_id, num in doc:
            prob_word = 0.0 # the probablity of the word 
            doc_word_num += num
            word = dictionary[word_id]
            for topic_id in range(num_topics):
                # cal p(w) : p(w) = sumz(p(z)*p(w|z))
                prob_topic = doc_topics_list[i][topic_id][1]
                prob_topic_word = topic_word_list[topic_id][word]
                prob_word += prob_topic*prob_topic_word
            prob_doc += math.log(prob_word) # p(d) = sum(log(p(w)))
        prob_doc_sum += prob_doc
        testset_word_num += doc_word_num
    prep = math.exp(-prob_doc_sum/testset_word_num) # perplexity = exp(-sum(p(d)/sum(Nd))
    print ("the perplexity of this ldamodel is : %s"%prep)
    return prep

for i in range(2,6):
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=i)
    prep = perplexity(lda, corpus, dictionary, len(dictionary.keys()), i)
    print(prep)

the info of this ldamodel: 

num of testset: 12; size_dictionary: 46; num of topics: 2
the perplexity of this ldamodel is : 5.451013346763121
5.451013346763121
the info of this ldamodel: 

num of testset: 12; size_dictionary: 46; num of topics: 3
the perplexity of this ldamodel is : 4.572624478331721
4.572624478331721
the info of this ldamodel: 

num of testset: 12; size_dictionary: 46; num of topics: 4
the perplexity of this ldamodel is : 4.205108041636469
4.205108041636469
the info of this ldamodel: 

num of testset: 12; size_dictionary: 46; num of topics: 5
the perplexity of this ldamodel is : 3.8637134513498252
3.8637134513498252
